In [1]:
###########################################################
#                                                         #
#               TRAITEMENT SIMUS FORTRAN                  #
#                     STATISTIQUES                        #
#                                                         #
#                         avec                            #
#                   - lecture arborescence                #
#                  analyse                                #
#                   - Delta Fluo                          #
#                   - Delta Energy GMol                   #
#                   - R/L Ca+                             #
#                   - distance inter ionique              #
#                   - trajectoire GMol                    #
#                                                         #
#                          02/2020     Adrien Poindron    #
###########################################################

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Imports

In [2]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

matplotlib.rcParams.update({'font.size': 21})
# matplotlib.rcParams["figure.dpi"] = 300

from IPython.display import display
from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

from scipy.optimize import curve_fit

In [3]:
import os
from os import listdir
from os.path import isfile, join

import tkinter as tk
from tkinter import filedialog
%matplotlib qt

from pathlib import Path
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [4]:
from natsort import natsorted   # natural sort marche de ouf
import re                       # find '/' in strings
import timeit
cm = pylab.get_cmap('summer')   # colormap par défaut color=cm(k/len(array))

In [5]:
# fonction Drewsen wz/wx en fct de alphaRL
def alpha_dre(alpha):
    alpha1=sqrt(-2*(arcsin((1-alpha**(-2))**0.5)-alpha*(1-alpha**(-2))**0.5)/(arcsin((1-alpha**(-2))**0.5)-alpha**-1*(1-alpha**(-2))**0.5))
    alpha0=sqrt(-2*(arcsinh((alpha**(-2)-1)**0.5)-alpha*(alpha**(-2)-1)**0.5)/(arcsinh((alpha**(-2)-1)**0.5)-alpha**-1*(alpha**(-2)-1)**0.5))
    return alpha0,alpha1

def alpha_R(R,swx,kappa):
    e=1.602e-19
    m=40.078*1.66054e-27 # 40.078
    eps0 = 8.854187*1e-12
    rho=2*m*eps0*swx**2/e**2
    U0=m*d0**2*((swx**2*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[0])/(1+0.5*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[0]))/(2*kappa*e)
    U1=m*d0**2*((swx**2*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[1])/(1+0.5*alpha_dre((R/1000)**3*4*pi*rho/(3*N))[1]))/(2*kappa*e)
    return U0,U1

In [6]:
def tick_function_q(my_q):
    V = my_q * 107.69 # 107.69123454143177
    return V

def tick_function_urf(my_Urf):
    my_q =  my_Urf / 107.69123454143178
    return my_q

def tick_function_a(my_a):
    V = my_a * 149.83128284025287
    return V

def tick_function_udc(Uend):
    aaz = Uend * 0.006674173650813496
    return aaz

In [7]:
# Fonctions persos de Jofre et Adrien

# %run function_jofre.ipy
%run "../../Functions/0.2/data_fct_Adrien_point_by_point.py"

In [8]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.05e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


# Sélection fichier

In [9]:
## GUI for data loading
# Select one data file all the way down to the directories
# Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+00V_D1.0_S1.0RFG.dat

file_cfg, slash_cfg, all_subdir = load_gui(filter_nocomplete=1)

file_path = file_cfg[0]
dir_path = file_cfg[1]
work_rep = file_cfg[2]
filename = file_cfg[3]

myslashpos = slash_cfg[0]
slashcond = slash_cfg[1]

> Répertoire : J:/20200131/DC01_RF08/Try00
> Filename : Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat
> myslashpos | [2, 11, 21]
> slashcond | -2
Points deleted because they were not complete ['DC10_RF18']   10 pt(s)
Total number of data directories 832


In [10]:
## Condition for data
points_and_coord, condition_parameters = simu_conditions(all_subdir, myslashpos, slashcond, filename)

> condition names ['DC', 'RF']
> number of points 64
000 > DC01_RF08
001 > DC01_RF09
002 > DC01_RF10
003 > DC01_RF11
004 > DC01_RF14
005 > DC01_RF15
006 > DC01_RF16
007 > DC01_RF17
008 > DC01_RF18
009 > DC03_RF08
010 > DC03_RF09
011 > DC03_RF10
012 > DC03_RF11
013 > DC03_RF14
014 > DC03_RF15
015 > DC03_RF16
016 > DC03_RF17
017 > DC03_RF18
018 > DC05_RF08
019 > DC05_RF09
020 > DC05_RF10
021 > DC05_RF11
022 > DC05_RF14
023 > DC05_RF15
024 > DC05_RF16
025 > DC05_RF17
026 > DC05_RF18
027 > DC07_RF08
028 > DC07_RF09
029 > DC07_RF10
030 > DC07_RF11
031 > DC07_RF14
032 > DC07_RF15
033 > DC07_RF16
034 > DC07_RF17
035 > DC07_RF18
036 > DC09_RF08
037 > DC09_RF09
038 > DC09_RF10
039 > DC09_RF11
040 > DC09_RF14
041 > DC09_RF15
042 > DC09_RF16
043 > DC09_RF17
044 > DC09_RF18
045 > DC10_RF08
046 > DC10_RF09
047 > DC10_RF10
048 > DC10_RF11
049 > DC10_RF14
050 > DC10_RF15
051 > DC10_RF16
052 > DC10_RF17
053 > DC11_RF08
054 > DC11_RF09
055 > DC11_RF10
056 > DC11_RF11
057 > DC12_RF10
058 > DC13_RF10
059

In [11]:
## Data loading (1D arrays)
data_name, num_runs, PMandT, Gmol_data, r_LC_clip, dim_nu \
= data_retrieve(all_subdir, points_and_coord,condition_parameters,slash_cfg) 

data0 = data_name[0]
data2 = data_name[1]
data4 = data_name[2]

PMvar = PMandT[0]
Tvar = PMandT[1]

deltaEc = Gmol_data[0]
deltaEcRel = Gmol_data[1]
t_c = Gmol_data[2]

> Points | 64
> Simulations pour chaque point | ['Try00', 'Try01', 'Try02', 'Try03', 'Try04', 'Try05', 'Try06', 'Try07', 'Try08', 'Try09', 'Try10', 'Try11', 'Try12']
Hello
Point n° 0
00 - 00  >  J:/20200131\DC01_RF08\Try00/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 01  >  J:/20200131\DC01_RF08\Try01/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 02  >  J:/20200131\DC01_RF08\Try02/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 03  >  J:/20200131\DC01_RF08\Try03/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 04  >  J:/20200131\DC01_RF08\Try04/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 05  >  J:/20200131\DC01_RF08\Try05/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 06  >  J:/20200131\DC01_RF08\Try06/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 07  >  J:/20200131\DC01_RF08\Try07/Temp_SimuType0_N01024_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG
00 - 08  >  J:/20200131\DC01_RF08

06 - 04  >  J:/20200131\DC01_RF16\Try04/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 05  >  J:/20200131\DC01_RF16\Try05/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 06  >  J:/20200131\DC01_RF16\Try06/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 07  >  J:/20200131\DC01_RF16\Try07/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 08  >  J:/20200131\DC01_RF16\Try08/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 09  >  J:/20200131\DC01_RF16\Try09/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 10  >  J:/20200131\DC01_RF16\Try10/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 11  >  J:/20200131\DC01_RF16\Try11/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
06 - 12  >  J:/20200131\DC01_RF16\Try12/Temp_SimuType0_N01024_Vrf0066_Udc0.1000D+01V_D1.0_S1.0RFG
Point n° 7
07 - 00  >  J:/20200131\DC01_RF17\Try00/Temp_SimuType0_N01024_Vrf0068_Udc0.1000D+01V_D1.0_S1.0RFG
07 - 01  

12 - 09  >  J:/20200131\DC03_RF11\Try09/Temp_SimuType0_N01024_Vrf0070_Udc0.2000D+01V_D1.0_S1.0RFG
12 - 10  >  J:/20200131\DC03_RF11\Try10/Temp_SimuType0_N01024_Vrf0070_Udc0.2000D+01V_D1.0_S1.0RFG
12 - 11  >  J:/20200131\DC03_RF11\Try11/Temp_SimuType0_N01024_Vrf0070_Udc0.2000D+01V_D1.0_S1.0RFG
12 - 12  >  J:/20200131\DC03_RF11\Try12/Temp_SimuType0_N01024_Vrf0070_Udc0.2000D+01V_D1.0_S1.0RFG
Point n° 13
13 - 00  >  J:/20200131\DC03_RF14\Try00/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.0_S1.0RFG
13 - 01  >  J:/20200131\DC03_RF14\Try01/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.0_S1.0RFG
13 - 02  >  J:/20200131\DC03_RF14\Try02/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.0_S1.0RFG
13 - 03  >  J:/20200131\DC03_RF14\Try03/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.0_S1.0RFG
13 - 04  >  J:/20200131\DC03_RF14\Try04/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.0_S1.0RFG
13 - 05  >  J:/20200131\DC03_RF14\Try05/Temp_SimuType0_N01024_Vrf0061_Udc0.2000D+01V_D1.0_S1.0RFG
13 - 06 

19 - 01  >  J:/20200131\DC05_RF09\Try01/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 02  >  J:/20200131\DC05_RF09\Try02/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 03  >  J:/20200131\DC05_RF09\Try03/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 04  >  J:/20200131\DC05_RF09\Try04/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 05  >  J:/20200131\DC05_RF09\Try05/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 06  >  J:/20200131\DC05_RF09\Try06/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 07  >  J:/20200131\DC05_RF09\Try07/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 08  >  J:/20200131\DC05_RF09\Try08/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 09  >  J:/20200131\DC05_RF09\Try09/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 10  >  J:/20200131\DC05_RF09\Try10/Temp_SimuType0_N01024_Vrf0059_Udc0.3000D+01V_D1.0_S1.0RFG
19 - 11  >  J:/20200

25 - 06  >  J:/20200131\DC05_RF17\Try06/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
25 - 07  >  J:/20200131\DC05_RF17\Try07/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
25 - 08  >  J:/20200131\DC05_RF17\Try08/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
25 - 09  >  J:/20200131\DC05_RF17\Try09/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
25 - 10  >  J:/20200131\DC05_RF17\Try10/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
25 - 11  >  J:/20200131\DC05_RF17\Try11/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
25 - 12  >  J:/20200131\DC05_RF17\Try12/Temp_SimuType0_N01024_Vrf0068_Udc0.3000D+01V_D1.0_S1.0RFG
Point n° 26
26 - 00  >  J:/20200131\DC05_RF18\Try00/Temp_SimuType0_N01024_Vrf0071_Udc0.3000D+01V_D1.0_S1.0RFG
26 - 01  >  J:/20200131\DC05_RF18\Try01/Temp_SimuType0_N01024_Vrf0071_Udc0.3000D+01V_D1.0_S1.0RFG
26 - 02  >  J:/20200131\DC05_RF18\Try02/Temp_SimuType0_N01024_Vrf0071_Udc0.3000D+01V_D1.0_S1.0RFG
26 - 03 

...

38 - 03  >  J:/20200131\DC09_RF10\Try03/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 04  >  J:/20200131\DC09_RF10\Try04/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 05  >  J:/20200131\DC09_RF10\Try05/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 06  >  J:/20200131\DC09_RF10\Try06/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 07  >  J:/20200131\DC09_RF10\Try07/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 08  >  J:/20200131\DC09_RF10\Try08/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 09  >  J:/20200131\DC09_RF10\Try09/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 10  >  J:/20200131\DC09_RF10\Try10/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 11  >  J:/20200131\DC09_RF10\Try11/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
38 - 12  >  J:/20200131\DC09_RF10\Try12/Temp_SimuType0_N01024_Vrf0064_Udc0.5000D+01V_D1.0_S1.0RFG
Point n° 39
39 - 00 

44 - 08  >  J:/20200131\DC09_RF18\Try08/Temp_SimuType0_N01024_Vrf0071_Udc0.5000D+01V_D1.0_S1.0RFG
44 - 09  >  J:/20200131\DC09_RF18\Try09/Temp_SimuType0_N01024_Vrf0071_Udc0.5000D+01V_D1.0_S1.0RFG
44 - 10  >  J:/20200131\DC09_RF18\Try10/Temp_SimuType0_N01024_Vrf0071_Udc0.5000D+01V_D1.0_S1.0RFG
44 - 11  >  J:/20200131\DC09_RF18\Try11/Temp_SimuType0_N01024_Vrf0071_Udc0.5000D+01V_D1.0_S1.0RFG
44 - 12  >  J:/20200131\DC09_RF18\Try12/Temp_SimuType0_N01024_Vrf0071_Udc0.5000D+01V_D1.0_S1.0RFG
Point n° 45
45 - 00  >  J:/20200131\DC10_RF08\Try00/Temp_SimuType0_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
45 - 01  >  J:/20200131\DC10_RF08\Try01/Temp_SimuType0_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
45 - 02  >  J:/20200131\DC10_RF08\Try02/Temp_SimuType0_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
45 - 03  >  J:/20200131\DC10_RF08\Try03/Temp_SimuType0_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
45 - 04  >  J:/20200131\DC10_RF08\Try04/Temp_SimuType0_N01024_Vrf0053_Udc0.6000D+01V_D1.0_S1.0RFG
45 - 05 

Point n° 51
51 - 00  >  J:/20200131\DC10_RF16\Try00/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 01  >  J:/20200131\DC10_RF16\Try01/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 02  >  J:/20200131\DC10_RF16\Try02/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 03  >  J:/20200131\DC10_RF16\Try03/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 04  >  J:/20200131\DC10_RF16\Try04/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 05  >  J:/20200131\DC10_RF16\Try05/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 06  >  J:/20200131\DC10_RF16\Try06/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 07  >  J:/20200131\DC10_RF16\Try07/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 08  >  J:/20200131\DC10_RF16\Try08/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 09  >  J:/20200131\DC10_RF16\Try09/Temp_SimuType0_N01024_Vrf0066_Udc0.6000D+01V_D1.0_S1.0RFG
51 - 10 

57 - 05  >  J:/20200131\DC12_RF10\Try05/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 06  >  J:/20200131\DC12_RF10\Try06/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 07  >  J:/20200131\DC12_RF10\Try07/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 08  >  J:/20200131\DC12_RF10\Try08/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 09  >  J:/20200131\DC12_RF10\Try09/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 10  >  J:/20200131\DC12_RF10\Try10/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 11  >  J:/20200131\DC12_RF10\Try11/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
57 - 12  >  J:/20200131\DC12_RF10\Try12/Temp_SimuType0_N01024_Vrf0064_Udc0.7500D+01V_D1.0_S1.0RFG
Point n° 58
58 - 00  >  J:/20200131\DC13_RF10\Try00/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.0_S1.0RFG
58 - 01  >  J:/20200131\DC13_RF10\Try01/Temp_SimuType0_N01024_Vrf0064_Udc0.3330D+01V_D1.0_S1.0RFG
58 - 02 

63 - 10  >  J:/20200131\DC19_RF10\Try10/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.0_S1.0RFG
63 - 11  >  J:/20200131\DC19_RF10\Try11/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.0_S1.0RFG
63 - 12  >  J:/20200131\DC19_RF10\Try12/Temp_SimuType0_N01024_Vrf0064_Udc0.6500D+01V_D1.0_S1.0RFG
Time elapsed:  668.329522 s
Time elapsed:  11.138825366666667 m


In [12]:
# Where SNR for eeach try in each  are compared to a threshold
# If SNR > some_threshold , we count it as a detection
# Then for each point the ratio number_of_detection/number_of_try is calculated
# This detection efficiency 

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))
trig_detection = np.zeros((shapevar))
for i in range(len(points_and_coord)):
    SNR = Gmol_data[2][i]
    for k,m in enumerate(SNR):
        if m > SNR_trig_threshold:
            trig_detection[i,k] = 1

print(mean(trig_detection,axis=1))

print(len(trig_detection))

[0.46153846 1.         1.         0.         1.         1.
 1.         1.         0.         0.15384615 1.         1.
 0.         1.         1.         1.         1.         0.38461538
 0.         0.07692308 0.92307692 0.         0.46153846 0.84615385
 1.         1.         0.92307692 0.         0.         0.46153846
 1.         0.         0.46153846 0.53846154 1.         0.15384615
 0.         0.         0.38461538 1.         0.23076923 0.38461538
 0.23076923 0.76923077 0.53846154 0.         0.         0.69230769
 0.61538462 0.46153846 0.76923077 0.84615385 0.76923077 0.
 0.         0.30769231 1.         0.07692308 0.76923077 0.69230769
 0.38461538 0.46153846 0.61538462 0.92307692]
64


In [13]:
# Where is written Detuning and how much it is
temp_pos = filename.find('0RFG')
Detuning = filename[temp_pos-7:temp_pos-4]

# Plots
Nuage d'ions (XYZ)

## Visualise one specific point, and one case

In [14]:
# Choose point to plot

plot_point = 11
print(list(points_and_coord.keys())[plot_point]) # point condition
plot_try = 7
plot_subdir = all_subdir[plot_point*13+plot_try%13] # point address
print(plot_subdir)

DC03_RF10
J:/20200131\DC03_RF10\Try07


In [255]:
# plot ion cloud XYZ

print(plot_subdir)
onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
file_name = '{}/xva{}'.format(plot_subdir,sort(onlyfiles)[0].strip('.dat')[4:])
plot_XYZ(file_name,fig_name='XYZ_point'+str(plot_point)+'_try'+str(plot_try),fig_title='XYZ')

J:/20200131\DC03_RF10\Try07


In [15]:
# Plot T and fluo PM

onlyfiles = [f for f in listdir(plot_subdir) if isfile(join(plot_subdir, f)) and not "xva" in f and ".dat" in f]
ta, te, ti, to, tu = plot_T_and_PM_Init_Inje_Evol( plot_subdir+'/',sort(onlyfiles)[0].strip('.dat')[4:],1,'coucou')
# ,xlim1=(3.458,3.582),ylim1=(2e-4,50),ylim2=(19.5,65)

In [257]:
# Plot only T with simulation time step
fname = 'only T'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti)
ax.grid()

ax.set_xlabel('[ms]')
ax.set_ylabel('[cts/500ns]')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

In [25]:
# Plot only T with custom time step (realistic PM behaviour)

time_step = 1e-3 # s
custom_pts = len(ta)*(ta[1]-ta[0])/time_step
print(custom_pts)
somme_pts = int( round( time_step/(ta[1]-ta[0]) ) )
print(somme_pts)

custom_t = np.linspace(1,ceil(max(ta)/time_step),custom_pts+1)
custom_t_offset = max(custom_t) - max(ta)/time_step
custom_t = custom_t - custom_t_offset
custom_fluo = np.zeros( int(ceil(custom_pts)) )

for j in range( int(ceil(custom_pts))-1 ):
    temp = []
    print(j)
    print(somme_pts*j,somme_pts*(j+1)-1)
    for k in range(somme_pts*j,somme_pts*(j+1)):
        temp.append(ti[-k-1])
    temp = sum(temp)
    custom_fluo[-1-j] = temp/max(ti)/15

fname = 'only T custom T step'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
ax.plot(ta*1000,ti,zorder=10,label='500 ns')
# for j in range( int(ceil(custom_pts)) ):
#     ax.plot(ta[::-1][somme_pts*j:somme_pts*(j+1)]*1000,ti[::-1][somme_pts*j:somme_pts*(j+1)])
ax.scatter(custom_t[1:],custom_fluo[1:]/max(custom_fluo)*max(ti),s=200,marker='+',lw=5,edgecolor='black',zorder=100,label='1 ms')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax.grid()


ax.set_xlim(1.1,5.98)
ax.set_xlabel('[ms]')
ax.set_ylabel('[cts]')
ax.legend(title='Time step')
tight_layout()
 
# savefig('ex_simu_clic_tr6_dc4_rf5_zoom.png',dpi=300)

5.9323951219512825
2030


TypeError: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.

## Data visualisation for all points (SNR)

In [24]:
# Voltages associated with each condition
# In names and files each voltage is associated with numbers
# For instance on epoint is labeled DC03_RF18
# You need to know which voltage is DC03 and RF18

# You have to look on the Fortran code, example :
#    !dec$ elseif defined(Vrf18)
#    double precision  , parameter :: V_rf  = 7.179d1
#    character(len=100), parameter :: str_extra3 = 'q=0.666'
# Here RF18 is 71.79 V, q = 0.666

Udc = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 6, 7, 7.5, 3.33, 3.66, 4.5, 5.33, 5.66, 6, 6.5]
Urf = [10.77, 16.15, 21.54, 26.92, 32.31, 37.69, 43.08, 48.46, 53.85, 59.23, 64.61,
       70.00, 75.38, 80.77, 61.02, 62.82, 66.41, 68.20, 71.79, 55.64, 57.43]
len(Udc),len(Urf)

(20, 21)

In [25]:
# A 3D visualisation of SNR for each point, each run
# Compulsory because this is where SNR is put in shape (under the name Z)

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]

fname = '3D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111, projection='3d')

# fig1 = plt.figure(num=fname,clear=True)
# ##### PLOT DFluo et Proba détection
# ax1 = fig1.add_subplot(111)
# ax2 = ax1.twinx()
# clf()

l = 0
plot_Udc = []
plot_Urf = []
Z = []
for i,j in enumerate(points_and_coord):
    plot_Udc.append(Udc[int(points_and_coord[j][0])])
    plot_Urf.append(Urf[int(points_and_coord[j][1])])
    Z.append(Gmol_data[2][i])
            # ax.set_aspect('equal')
Z = np.array(Z)
print(shape(plot_Udc),shape(plot_Urf),shape(Z))

for k in range(13):
    ax.scatter(xs=plot_Udc,ys=plot_Urf,zs=Z[:,k],color='xkcd:azul',marker='o',s=10)
#             ax.plot(xs=r_LC_clip[0],ys=r_LC_clip[2],zs=r_LC_clip[1],marker='o',ms=0.45,ls='',color='xkcd:blue with a hint of purple')

#             ax.view_init(elev=18, azim=55)
#             ax.set_xlim(-3e-1,3e-1)
#             ax.set_ylim(-3e-1,3e-1)
#             ax.set_zlim(-3e-1,3e-1)

#             ax.text2D(0.05, 0.85, f'$U_{{ RF }}$ =  {Urf} V\n$U_{{DC}}$ = {Udc[j]} V', transform=ax.transAxes)
ax.set_xlabel('\n$U_{DC}$ [V]')
ax.set_ylabel('\n$U_{RF}$ [V]')
ax.set_zlabel('\n SNR ')

#         ax.plot(np.ones(100)*Udc[dc], np.ones(100)*Urf[rf], zs=linspace(0,0.8,100),color='black',ls=':',lw=1)

# X = [Udc for hhh in range(elem_1+1)]
# Y = [Urf for smeuiofh in range(elem_2-1)]
# # X, Y = np.meshgrid(X, Y)
# Z = asarray(transpose(mean_fluo))
# Z = np.meshgrid(Z)

# # surf = ax.plot_wireframe(transpose(X), Y, Z)
# surf = ax.plot_surface(transpose(X), Y, Z, rstride=1, cstride=1,
#                 cmap='viridis', edgecolor='black',alpha = 0.35)
# #         ax.errorbar(Udc[dc],mean_fluo[],yerr=prob_detect_xerr,xerr=None,
# #              elinewidth=2.5,capsize=None,color='xkcd:yellow orange',
# #              fmt=':+',ms=12,mew=3.5,mfc='None',mec='xkcd:yellow orange')
# cbar = plt.colorbar(surf)
# cbar.ax.set_ylabel(r'$\Delta F / F$ moyen ('+str(elem_0)+' essais)', rotation=90)


fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
# subplots_adjust(hspace=0.015)

# savefig(fname,dpi=300)

(64,) (64,) (64, 13)


## 2D plot

In [26]:
for i,j in enumerate(points_and_coord):
    print(j)
    print(points_and_coord[j][0],points_and_coord[j][1])

DC01_RF08
01 08
DC01_RF09
01 09
DC01_RF10
01 10
DC01_RF11
01 11
DC01_RF14
01 14
DC01_RF15
01 15
DC01_RF16
01 16
DC01_RF17
01 17
DC01_RF18
01 18
DC03_RF08
03 08
DC03_RF09
03 09
DC03_RF10
03 10
DC03_RF11
03 11
DC03_RF14
03 14
DC03_RF15
03 15
DC03_RF16
03 16
DC03_RF17
03 17
DC03_RF18
03 18
DC05_RF08
05 08
DC05_RF09
05 09
DC05_RF10
05 10
DC05_RF11
05 11
DC05_RF14
05 14
DC05_RF15
05 15
DC05_RF16
05 16
DC05_RF17
05 17
DC05_RF18
05 18
DC07_RF08
07 08
DC07_RF09
07 09
DC07_RF10
07 10
DC07_RF11
07 11
DC07_RF14
07 14
DC07_RF15
07 15
DC07_RF16
07 16
DC07_RF17
07 17
DC07_RF18
07 18
DC09_RF08
09 08
DC09_RF09
09 09
DC09_RF10
09 10
DC09_RF11
09 11
DC09_RF14
09 14
DC09_RF15
09 15
DC09_RF16
09 16
DC09_RF17
09 17
DC09_RF18
09 18
DC10_RF08
10 08
DC10_RF09
10 09
DC10_RF10
10 10
DC10_RF11
10 11
DC10_RF14
10 14
DC10_RF15
10 15
DC10_RF16
10 16
DC10_RF17
10 17
DC11_RF08
11 08
DC11_RF09
11 09
DC11_RF10
11 10
DC11_RF11
11 11
DC12_RF10
12 10
DC13_RF10
13 10
DC14_RF10
14 10
DC15_RF10
15 10
DC16_RF10
16 10
DC17_RF1

In [73]:
x = []
y = []
z = []
for g,h in enumerate(points_and_coord):
    x.append(Udc[int(points_and_coord[h][0])])
    y.append(Urf[int(points_and_coord[h][1])])
    z.append(mean(trig_detection[g]))
    
fname = '2D_GMol_var_fluo_contour'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=12)

im = ax.scatter(x,y,
        marker='o',s=100,edgecolor='black',
        c = z, cmap='rainbow')
ax.set_xlabel(r'$U_{DC}$ [V]')
ax.set_ylabel(r'$U_{RF}$ [V]')
ax.grid()

cbar = plt.colorbar(im, ax=ax)
cbar.ax.set_ylabel(r'Detection rate (SNR > '+str(SNR_trig_threshold)+ ', Detuning -'+ str(Detuning) +   ')\n ('+str(len(num_runs))+' runs for each point)', rotation=90)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

In [1212]:
fname = 'art'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)

im = ax.contourf([1,2,3],[1,2,3],[[1,2,3],[4,5,6],[7,8,9]]) # [[nan,2,nan],[4,5,6],[nan,8,nan]]
# for rf in range(1,4):
#     for dd in range(1,4):
#         ax.plot(rf,dd,marker='+',ls='',markersize=10,mew=3,color = 'xkcd:yellow orange')

# ax.set_zlabel(r'$\Delta F / F$')
# im2 = ax.contour(Udc, Urf, transpose(Z), colors='k')
ax.set_xlim(0.95,3.05)
ax.set_ylim(0.95,3.05)
fig.set_size_inches(11,8)
fig.tight_layout()
# savefig(fname+str(40),dpi=300)

## More specific visualisations

In [74]:
# Choose a specific condition to consider
# Choose one line on the plot (one RF)
condition_to_respect = 'RF10'
points_to_plot = [k for k,l in enumerate(list(points_and_coord.keys())) if condition_to_respect in l]
print(points_to_plot)

[2, 11, 20, 29, 38, 47, 55, 57, 58, 59, 60, 61, 62, 63]


In [75]:
# plot delta E Gmol absolu for one RF voltage

SNR_trig_threshold = 1
shapevar = ((len(points_and_coord), len(num_runs)))

ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]
alpha_dre = zeros(shapevar)
rtoLratio = zeros(shapevar)

fname = 'Fluo_stat_all'
fig1 = plt.figure(num=fname,clear=True)
gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(gs[0])
ax2 = ax1.twinx()
# clf()

DC_index = list(points_and_coord)[0].find('DC')
RF_index = list(points_and_coord)[0].find('RF')
plot_Udc = [Udc[ int(list(points_and_coord)[x][DC_index+2:DC_index+4])] for x in points_to_plot]
plot_Urf = [Urf[ int(list(points_and_coord)[x][RF_index+2:RF_index+4])] for x in points_to_plot]

suptitle(all_subdir[0][:-15]+'  | Urf='+str(plot_Urf[0])+', D=-'+str(Detuning),fontsize=12)

# az.append(8*mkappa*(C_e/m_Ca)*plot_Udc/(d0**2*Omega**2))
# qx.append(-4*(C_e/m_Ca)*plot_Urf/(r0**2*Omega**2))

#     # frequences attendues
#     wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
# #     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
#     wz.append(sqrt(2*mkappa*C_e*plot_Udc[i]/(m_Ca*d0**2))/(2*pi))
    
for i,j in enumerate(points_to_plot):
    # a,q attendu       
    SNR = Gmol_data[2][j]
    ax1.plot([plot_Udc[i]]*len(SNR),SNR,'o',color='xkcd:azul',markerfacecolor='none')
    
    prob_detect_xerr = sqrt(mean(trig_detection,axis=1)[j]*(1-mean(trig_detection,axis=1)[j]))/sqrt(len(SNR))
    ax2.errorbar(plot_Udc[i],mean(SNR),yerr=prob_detect_xerr,xerr=None,
             elinewidth=1,capsize=None,color='xkcd:yellow orange',
             fmt=':+',ms='8',mew=2,mfc='None',mec='xkcd:yellow orange')
#     annotate('qx={:1.1e}\nUrf={:3.1f}V\nwx={:3.1f}kHz'.format(qx[0],Urf_no_repeat,wx[0]/1000), xy=(2,0.25), xycoords='data',
#         size=10, ha='center', va='top', color='xkcd:azul',
#         bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul'))
    
    l+=1
    
# for k in range(0,len(Udc_no_repeat),3):
#         ax1.annotate('a={:1.1e}\nwz={:3.1f}kHz'.format(az[k],wz[k]/1000), xy=(Udc[k],0.73), xycoords='data',
#         size=8, ha='center', va='top', color='xkcd:azul',
#         bbox=dict(boxstyle='round', fc='none',edgecolor='none')) 
       
ax1.set_ylabel('SNR', color='xkcd:azul')
# ax2.set_ylabel('$P_{Detect}$'+' ('+str(elem_0)+' runs)', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')


ax1.set_ylim(-0.2,5.35)
ax2.set_ylim(-0.2,5.35)
ax1.grid()

##### Plot DEnergie et moyenne de DEnergie
renorm = 0.5*m_GM/C_e
ax3 = fig1.add_subplot(gs[2],sharex=ax1)
ax4 = ax3.twinx()
# clf()
for i,j in enumerate(points_to_plot):
    ax3.plot([plot_Udc[i]]*len(deltaEc[j][:]),deltaEc[j][:]*renorm*1000,'o',color='xkcd:azul',markerfacecolor='none')

mean_deltaEc = [mean(deltaEc[j][:])*renorm*1000 for i,j in enumerate(points_to_plot)]
        
ax4.plot(plot_Udc,mean_deltaEc,linestyle='',marker='o',color='xkcd:yellow orange')
ax4.annotate('$E_{GMol}$=50 eV', xy=(1,5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax3.set_xlabel('$U_{DC}$')
ax3.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax4.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax3.tick_params(axis='y', labelcolor='xkcd:azul')
ax4.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax3.set_ylim(-70,7)
ax4.set_ylim(-70,7)
ax3.grid()

ax3.set_xlim(-0.1,8.1)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

##### Plot R/L et alpha
ax5 = fig1.add_subplot(gs[1],sharex=ax1)
ax6 = fig1.add_subplot(gs[3],sharex=ax5)

rtoLratio = [[] for k in range(len(points_to_plot))]
for i,j in enumerate(points_to_plot):
    for k,l in enumerate(dim_nu[j,:,0]):
        rtoLratio[i].append( dim_nu[j,k,0] / dim_nu[j,k,2])
mean_rtoLratio = [mean(rtoLratio[i][:]) for i,j in enumerate(points_to_plot)]
for i,j in enumerate(points_to_plot):
    ax5.plot([plot_Udc[i]]*len(rtoLratio[i][:]),rtoLratio[i][:],'o',color='xkcd:azul',markerfacecolor='none')
#     ax6.plot(plot_Udc[i],alpha_dre[0][j],'o',color='xkcd:azul')
    
ax5.plot(plot_Udc,mean_rtoLratio,linestyle='',marker='o',color='xkcd:yellow orange',markerfacecolor='none')

ax6.set_xlabel('$U_{DC}$')
ax5.set_ylabel('R/L')
ax6.set_ylabel(r'$\alpha$')
# ax5.tick_params(axis='y', labelcolor='xkcd:azul')
# ax6.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax6.set_xlim(-0.1,8.1)
ax5.set_ylim(-0.05,0.6)
ax6.set_ylim(-1.65,0.35)
ax5.grid()
ax6.grid()

ax5.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
plt.setp(ax5.get_xticklabels(), visible=False)

tight_layout()

fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()
subplots_adjust(hspace=0.015,top=0.935)

savefig(fname,dpi=300)

In [34]:
ax = 0
ay = 0
az, qx, wx, wy, wz = [],[],[],[],[]
alpha_dre = zeros(shape(points_to_plot))

for i,j in enumerate(points_to_plot):
    az.append(8*mkappa*(C_e/m_Ca)*plot_Udc[i]/(d0**2*Omega**2))
    qx.append(-4*(C_e/m_Ca)*plot_Udc[i]/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[i]**2/2+ax-0.5*az[i])/(2*pi))
    #     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*C_e*plot_Udc[i]/(m_Ca*d0**2))/(2*pi))


for k,l in enumerate(plot_Urf): # try
    print(j,k,l)
    alpha_dre[k][j] = [j]**2/wx[j][l]**2

63 0 64.61


H:\Programmes\Anaconda3\envs\py307\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in sqrt
  # This is added back by InteractiveShellApp.init_path()


TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [113]:
print('===== Udc =====')
for a,b in enumerate(Udc):
    print(' <>',f'{a:02d}',':',f'{b:.2f}',end='')
print('\n===== Urf =====')
for a,b in enumerate(Urf):
    print(' <>',f'{a:02d}',':',f'{b:.2f}',end='')

===== Udc =====
 <> 00 : 0.50 <> 01 : 1.00 <> 02 : 1.50 <> 03 : 2.00 <> 04 : 2.50 <> 05 : 3.00 <> 06 : 3.50 <> 07 : 4.00 <> 08 : 4.50 <> 09 : 5.00 <> 10 : 6.00 <> 11 : 7.00 <> 12 : 7.50 <> 13 : 3.33 <> 14 : 3.66 <> 15 : 4.50 <> 16 : 5.33 <> 17 : 5.66 <> 18 : 6.00 <> 19 : 6.50
===== Urf =====
 <> 00 : 10.77 <> 01 : 16.15 <> 02 : 21.54 <> 03 : 26.92 <> 04 : 32.31 <> 05 : 37.69 <> 06 : 43.08 <> 07 : 48.46 <> 08 : 53.85 <> 09 : 59.23 <> 10 : 64.61 <> 11 : 70.00 <> 12 : 75.38 <> 13 : 80.77 <> 14 : 61.02 <> 15 : 62.82 <> 16 : 66.41 <> 17 : 68.20 <> 18 : 71.79 <> 19 : 55.64 <> 20 : 57.43

In [114]:
# Choose a specific point to consider
# Choose one point on the plot
condition_to_respect = 'DC03_RF10'
address_to_plot = [l for k,l in enumerate(all_subdir) if condition_to_respect in l]
points_to_plot  = [k for k,l in enumerate(list(points_and_coord.keys())) if condition_to_respect in l]
print(address_to_plot,points_to_plot)

['J:/20200131\\DC03_RF10\\Try00', 'J:/20200131\\DC03_RF10\\Try01', 'J:/20200131\\DC03_RF10\\Try02', 'J:/20200131\\DC03_RF10\\Try03', 'J:/20200131\\DC03_RF10\\Try04', 'J:/20200131\\DC03_RF10\\Try05', 'J:/20200131\\DC03_RF10\\Try06', 'J:/20200131\\DC03_RF10\\Try07', 'J:/20200131\\DC03_RF10\\Try08', 'J:/20200131\\DC03_RF10\\Try09', 'J:/20200131\\DC03_RF10\\Try10', 'J:/20200131\\DC03_RF10\\Try11', 'J:/20200131\\DC03_RF10\\Try12'] [11]


In [115]:
#y_hist_detec[i]
renorm = 0.5*m_GM/C_e
deltaEc[points_to_plot][0]*renorm*1000

array([-33.99788598, -39.59334315, -43.71237157, -29.094577  ,
       -26.99180728, -38.82423406, -36.49234823, -52.26303362,
       -34.2694627 , -30.45005438, -48.36314425, -44.52362659,
       -37.51380858])

In [116]:
# histogram for Delta energy
# two families : detection and non detection
y_hist_detec    = [l*renorm*1000 for l in deltaEc[points_to_plot][0] if abs(l) > SNR_trig_threshold]
y_hist_nondetec = [l*renorm*1000 for l in deltaEc[points_to_plot][0] if abs(l) < SNR_trig_threshold]

fig = plt.figure(num='Histogram Delta energy GMol', clear=True)
# plt.suptitle('Histogram $\Delta E$')
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1]) 
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex=ax1)
 # PMvar[k][my_selection]
my_bins = arange(-60,0,2)
ax1.set_title(r'Detection SNR>'+str(SNR_trig_threshold),color='xkcd:leaf green')
ax1.hist(y_hist_detec,bins=my_bins ,color='xkcd:leaf green', edgecolor='black', linewidth=1.2)
ax2.set_xlabel(r'$\Delta E$ [meV]')
ax1.set_ylim(0,4)
ax1.grid()
ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=2))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax1.annotate(f'{len(y_hist_detec)} cases', xy=(-60,3.5), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:leaf green',
    bbox=dict(boxstyle='round', fc='white')) 

ax2.set_title(r'No detection SNR>'+str(SNR_trig_threshold),color='xkcd:rouge')
ax2.hist(y_hist_nondetec,bins=my_bins ,color='xkcd:rouge', edgecolor='black', linewidth=1.2)
ax2.set_ylim(0,4)
ax2.grid()
ax2.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=1))
ax2.annotate(f'{len(y_hist_nondetec)} cases', xy=(-60,3.5), xycoords='data',
    size=12, ha='left', va='top', color='xkcd:rouge',
    bbox=dict(boxstyle='round', fc='white')) 

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.22,top=0.94)

# savefig('hist_simu.png', dpi=300)

In [1071]:
# plot DE vs Udc but with red green colors

# my_order = [0,1,2,3,4,5,6,13,7,8,9,14,15,10,16,11,12] # 13,7,8,9,14,15,10,16,11,12

# Udc = [0.5,1,1.5,2,2.5,3,3.5,3.75,4,4.5,5,5.3,5.7,6,6.5,7,8]
# Urf = 64.61

mean_fluo = [mean(PMvar[:,j]) for j in my_order]

fig1 = plt.figure(num='Fluo var only_DE_colors',clear=True)

##### PLOT DFluo et Proba détection
ax1 = fig1.add_subplot(111)
ax2 = ax1.twinx()

left, bottom, width, height = [0.4145, 0.12, 0.45, 0.5] # cadre zoom
axins = fig1.add_axes([left, bottom, width, height])

my_color = ['xkcd:rouge', 'xkcd:leaf green']
l = 0
for j in range(len(Udc)):          
    # a,q attendu
    az.append(8*mkappa*(e/m)*Udc[j]/(d0**2*Omega**2))
    qx.append(-4*(e/m)*Urf/(r0**2*Omega**2))

    # frequences attendues
    wx.append(Omega/2*sqrt(qx[l]**2/2+ax-0.5*az[l])/(2*pi))
#     wy.append(Omega/2*sqrt(qx[l]**2/2+ay-0.5*az[l])/(2*pi))
    wz.append(sqrt(2*mkappa*e*Udc[j]/(m*d0**2))/(2*pi))
    
    for k in range(elem_0):
        trig = 0
        if PMvar[k][my_order[j]] > 0.6:
            sigdetect[j] +=1
            trig = 1
        alpha_dre[k][j] = wz[j]**2/wx[j]**2
        rtoLratio[k][j] = dim_nu[k,j,0] / dim_nu[k,j,2]
        ax1.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
        axins.plot(Udc[j],deltaEc[k][my_order[j]]*renorm*1000,'o',color=my_color[trig],markerfacecolor='none')
    
    l+=1


mean_deltaEc = [mean(deltaEc[:,j])*renorm*1000 for j in my_order]
        
ax2.plot(Udc,mean_deltaEc,linestyle=':',marker='>',color='xkcd:yellow orange',markerfacecolor='none')
ax2.annotate('$E_{GMol}$=50 eV', xy=(1,-5), xycoords='data',
    size=10, ha='center', va='top', color='xkcd:azul',
    bbox=dict(boxstyle='round', fc='none',edgecolor='xkcd:azul')) 

ax1.set_xlabel('$U_{DC}$')
ax1.set_ylabel('$\Delta E$ [meV]', color='xkcd:azul')
ax2.set_ylabel('mean $\Delta E$ [meV]', color='xkcd:yellow orange')
ax1.tick_params(axis='y', labelcolor='xkcd:azul')
ax2.tick_params(axis='y', labelcolor='xkcd:yellow orange')

ax1.set_ylim(-0.10*1000,1)
ax2.set_ylim(-0.10*1000,1)
ax1.grid()

ax2.set_xlim(-0.1,8.5)

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
ax1.yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=10))

x1, x2, y1, y2 = 3.25, 8.25, -21, -2.5 # limites zoom
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.grid()

mark_inset(ax1, axins, loc1=2, loc2=1, fc="none", ec="0.4")
# axins.invert_yaxis()
plt.setp(axins.get_xticklabels(), visible=False)
plt.setp(axins.get_yticklabels(), fontsize=10)


fig1.set_size_inches(11.69, 8.27)
fig1.tight_layout()

/home/adrian/.local/lib/python3.6/site-packages/matplotlib/figure.py:2267: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "


# Trajectory
Trajectoire GMol

In [117]:
# Recover all GMol trajectories addresses
# + compare bi for-loop with nested comprehension list

# Soit A
trajectory_file = []
for j,k in enumerate(address_to_plot):
    temp_files = listdir(k)
    for l,m in enumerate(temp_files):
        if 'trj' in m:
            trajectory_file.append(m)

# Soit B
trajectory_files = [[m for l,m in enumerate(temp_files) if 'trj' in m] for j,k in enumerate(address_to_plot)]
    
#trajectory_file = [listdir(j) for i,j in enumerate(address_to_plot)]
#trajectory_file = [k for j,k in enumerate(trajectory_file) if 'trj' in k]
print(trajectory_file)

['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin', 'trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']


In [118]:
temp_size = len(load_trj(os.path.join(address_to_plot[0],trajectory_files[0][0]).strip('.bin'))[0])
t, r_x,r_y,r_z, v_x,v_y,v_z, a_x,a_y,a_z = np.zeros((10,len(trajectory_files),temp_size))
for j,k in enumerate(trajectory_files): # try
    print('>',j,'-',k)
    t[j], r_x[j],r_y[j],r_z[j], v_x[j],v_y[j],v_z[j], a_x[j],a_y[j],a_z[j] = load_trj(os.path.join(address_to_plot[j],k[0].strip('.bin')))

> 0 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 1 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 2 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 3 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 4 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 5 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 6 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 7 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 8 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 9 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 10 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 11 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S1.0RFG50eV.bin']
> 12 - ['trj_SimuType4_01_N01024_Vrf0064_Udc0.2000D+01V_D1.0_S

In [119]:
Ec_GMol = np.zeros((len(trajectory_files),temp_size))
for walay,bilai in enumerate(trajectory_files):
    for couscous,merguez in enumerate(t[walay]):
        Ec_GMol[walay,couscous] = 0.5*m_GM*(v_x[walay,couscous]**2+v_y[walay,couscous]**2+v_z[walay,couscous]**2)

In [140]:
v_z[walay,couscous]

97.91841226560392

In [157]:
# plot energy evolution of GMol
# during fly time
# depending time and space
cm = pylab.get_cmap('rainbow')   # colormap par défaut color=cm(k/len(array))

fname = 'GMol_energy_time'
fig = plt.figure(fname,clear=True)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
#title(all_subdir[0][:-15],fontsize=12)

for i in range(13):
    #i = 0
    w = t[i]
    x = r_z[i]
    y = Ec_GMol[i]
    cloud_size = dim_nu[points_to_plot,i,2]

    im = ax1.scatter(w[22:40]*1000,y[22:40]/C_e,
            marker='+',s=42,color=cm(i/12))
    ax1.set_xlabel(r't [ms]')
    ax1.set_ylabel(r'Energie [eV]')
    ax1.grid()

    im = ax2.scatter(x[22:40]*1000,y[22:40]/C_e,
            marker='+',s=42,color=cm(i/12))
    ax2.set_xlabel(r'r_z [mm]')
    ax2.set_ylabel(r'Energie [eV]')
    ax2.grid()

    ax2.vlines(-cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)
    ax2.vlines(cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)
# ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
#savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

In [163]:
cm = pylab.get_cmap('rainbow')   # colormap par défaut color=cm(k/len(array))

fname = 'GMol_energy_pos'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
#title(all_subdir[0][:-15],fontsize=12)

for i in range(13):
    w = t[i]
    x = r_z[i]
    y = Ec_GMol[i]
    cloud_size = dim_nu[points_to_plot,i,2]
    im = ax.scatter(x[22:40]*1000,y[22:40]/C_e,
            marker='+',s=55,lw=3,color=cm(i/13))
    ax.set_xlabel(r'r_z [mm]')
    ax.set_ylabel(r'Energie [??]')
    ax.grid()
    

ax.vlines(-cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)
ax.vlines(cloud_size,49.92,49.995,color='gray',linestyle=':',lw=2)       
    # ax.grid()
fig.set_size_inches(11.69, 8.27)

fig.tight_layout()
    # savefig(fname+'_D'+str(Detuning)+'.png',dpi=300)

#  Some calculations about physics

In [ ]:
# comparing amplitude RF mvmnt to Thermal mvmnt

to_calc = 4
T = 6e-4

Er = 2*Urf*mean(dim_nu[:,my_order[to_calc],0])/r0

RF_amp = C_e*Er / (m_Ca*Omega**2)
Th_amp = sqrt(kb*T**2/ (m_Ca*wx[my_order[to_calc]]**2) )

print(Th_amp, RF_amp)

In [164]:
frac_E0_Ecr = 50
wz_LC = 2*pi*90806.9982303
rz_0 = -1.5e-3

wz2 = wz_LC**2 * m_Ca / m_GM

L     = 0.014  #(demi-longueur piège obtenue avec SIMION)
eta = 1. / (4.*(sqrt(10) + 5.)) # (~0.0306 je soupçonne cette valeur d'être le facteur d'écrantage)

Ep0   = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )
Ek_init = C_e * frac_E0_Ecr - Ep0

print(frac_E0_Ecr,Ep0/C_e,Ek_init/C_e)

50 -0.16732941758411765 50.167329417584114


In [233]:
rz  = np.linspace(rz_0*6,-rz_0,1000)
# Ep = wz2*eta*L**2*m_GM*(1- exp(rz_0**2 / (2*eta*L**2)) )
my_cte = -1/(2*eta*L**2)
Ep = wz2*eta*L**2*m_GM*(1-exp(my_cte*rz**2))/C_e
Ep_real = (wz2*m_GM)*rz**2/C_e

fname = 'coucou'
fig = plt.figure(fname,clear=True)
ax = fig.add_subplot(111)
title(all_subdir[0][:-15],fontsize=12)

ax.plot(rz,Ep,label='Gaussian like for axial')
ax.plot(rz[600:],Ep_real[600:],label='Exact harmonic pot.')
ax.vlines(rz_0,0,1,label='Initial pos.')
ax.set_xlabel('z [mm]')
ax.set_ylabel('E_p [eV]')
ax.legend()

ax.grid()
tight_layout()

## Distance moyenne entre les ions Ca$^+$

In [1039]:
dist_ions = zeros(shapevar)
dist_ions_theo = zeros(shapevar)
V = zeros(shapevar)
V_theo = zeros(shapevar)
rho = zeros(shapevar)
k = 3
for k in range(elem_0):
    for j in range(elem_2):     # Udc
        rho[k][j] = 2*m*eps0/e**2*(wx[j]*2*pi)**2 # "théorique" on s'en fout
        V_theo[k][j] = N/rho[k][j]
        V[k][j] = 4/3*pi*dim_nu[k][my_order[j]][2]*1e-3 * ( dim_nu[k][my_order[j]][0]*1e-3 )**2
    #     rho[j] = N/V_moy[j] # donné par le volume et le nombre d'ions
    #     dist_ions[j] = ( 1 / (V_moy[j]*rho[j]) ) ** (-1/3) # en µm
        dist_ions[k][j] = (V[k][j]/N)**(1/3)
        dist_ions_theo[k][j] = (V_theo[k][j]/N)**(1/3)

In [1012]:
max_HC_trj = zeros((elem_0,elem_2,3))
for j in range(elem_2):     # Udc
    print('j - k')
    for k in range(elem_0): # try
        max_HC_trj[k,j,0] = max(abs( r_x[k][my_order[j]] ))
        max_HC_trj[k,j,1] = max(abs( r_y[k][my_order[j]] ))
        max_HC_trj[k,j,2] = max(abs( r_z[k][my_order[j]] ))

j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k
j - k


In [1073]:
fig = plt.figure(num='Dist ions et GMol',clear=True)
# gs = gridspec.GridSpec(2, 2, width_ratios=[8, 4])   gs[0]
##### PLOT DFluo et Proba détection
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212,sharex = ax1)
# ax3 = fig.add_subplot(gs[1])
# ax4 = fig.add_subplot(gs[3], projection='3d')

for j in range(elem_2):
    for k in range(elem_0):
        Vtmp = 4/3*pi*dim_nu[k][my_order[j]][2]*1e-3 * ( dim_nu[k][my_order[j]][0]*1e-3 )**2
        dist_ionstmp = (Vtmp/N)**(1/3)
        ax1.plot(Udc[j],dist_ionstmp*1e6,'o',color='xkcd:azul',markerfacecolor='none')
    ax1.plot(Udc[j],mean(dist_ions[:,j])*1e6,'o',marker='8',mew=2,mfc='None',mec='xkcd:yellow orange')
# ax1.set_xlabel(r'$U_{DC}$ [V]')
ax1.set_ylabel(r'$d_{moy}$ Ca$^+$ [µm]')
ax1.grid()

for j in range(elem_2):
    for k in range(elem_0):
        r_sortie = sqrt(max_HC_trj[k,j,0]**2 + max_HC_trj[k,j,1] **2 )
        ax2.plot(Udc[j],r_sortie*1e9,'o',color='xkcd:azul',markerfacecolor='none')
    r_sortie = sqrt(mean(max_HC_trj[:,j,0])**2 + mean(max_HC_trj[:,j,1]) **2 )
    ax2.plot(Udc[j],r_sortie*1e9,marker='8',mew=2,mfc='None',mec='xkcd:yellow orange')

#     ax2.plot(Udc[j],mean(max_HC_trj[:,j,1])*1e9,color='xkcd:azul',marker='+')
#     ax2.plot(Udc[j],mean(max_HC_trj[:,j,2])*1e9,color='xkcd:brown',marker='+')
ax2.set_xlabel(r'$U_{DC}$ [V]')
ax2.set_ylabel('r sortie max GMol [nm]')
ax2.grid()

ax1.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.5))
plt.setp(ax1.get_xticklabels(), visible=False)

fig.set_size_inches(11.69, 8.27)
fig.tight_layout()
subplots_adjust(hspace=0.015)

In [38]:
# Dimensions nuage
k = 3
j = 3
for k in range(63):
    print(k,dim_nu[k][j]*1e3)
print(mean(dim_nu[k][j][0])*1e3)
print(mean(dim_nu[k][j][1])*1e3)
print(mean(dim_nu[k][j][2])*1e3)

0 [ 43.732358    43.28602545 378.47651605]
1 [ 38.90063402  39.1069678  386.65188276]
2 [ 36.04323513  36.4331021  394.15014656]
3 [ 59.89196967  59.64205967 966.97057725]
4 [ 38.37931688  38.271461   389.45838538]
5 [ 36.31292115  36.19715829 391.72733568]
6 [ 33.87829886  34.25913873 396.30345831]
7 [ 33.11675824  33.31679148 398.0690542 ]
8 [ 59.92952978  59.91219623 985.55982094]
9 [ 53.27260747  52.45615797 272.18563196]
10 [ 47.25041812  47.19602193 280.70653239]
11 [ 42.95769517  42.65428235 288.1526562 ]
12 [ 59.18422837  59.78078959 944.58872577]
13 [ 45.64889732  45.40465801 283.50282553]
14 [ 44.16731482  44.21837088 285.60145806]
15 [ 42.03091072  41.11421926 289.75721414]
16 [ 39.75591941  39.67749157 292.06519641]
17 [ 37.75465543  38.12341683 295.76420861]
18 [ 59.79765025  59.89441941 218.79903141]
19 [ 53.1065669   52.41087309 228.49110186]
20 [ 48.12423555  47.70368657 235.64672856]
21 [ 59.92966942  59.6485581  944.49193495]
22 [ 50.82715686  51.12118777 228.84711558

In [39]:
dist_ions*1e6

NameError: name 'dist_ions' is not defined

In [736]:
dist_ions_theo * 1e6

array([14.33309411, 14.37808383, 14.42364379, 14.46978675, 14.51652592,
       14.56387488, 14.61184771, 14.6360726 , 14.66045894, 14.70972358,
       14.75965715, 14.78994505, 14.83071866, 14.86159591, 14.91363491,
       14.96641052, 15.07424603])

# Ecriture fichier
Garder les principales variables dans un fichier texte

In [898]:
float_formatter = lambda x: '{:.04}'.format(x)

In [83]:
# name=['Uend ', 'ax   ','az   ','qx   ','kappa','Udc  ','Urf  ','aax  ','aaz  ','aqx  ','awx  ','awz  ','aUdc ','aUend']
# var=[Uend, ax, az, qx, kappa, Udc, Urf, aax, aaz, aqx, awx, awz, aUdc, aUend]


f=open("simu_variables.txt","w+")
f.write('Python3.7 %s\n' % datetime.datetime.now())
f.write('Simulations collisions GMol -> Ca+ \n\n\n')

f.write('> Répertoire de travail\n')
f.write('%s\n\n' % work_rep)

f.write('> Conditions de simulation\n')
f.write(f'N_ions E_GMol\n')
f.write(f'{N} {e_GMol:>4}eV\n\n')

f.write(f'> Nombre total de simulations : {cond_number}\n')
f.write(f'{cond_two_name} : {elem_2} élément(s)\n')
f.write(f'{[ int(x) for x in cond_two]}\n')
f.write(f'{[ Udc[x] for x in my_order ]} V\n')
f.write(f'{cond_one_name} : {elem_1} élément(s)\n')
f.write(f'{[ int(x) for x in cond_one]}\n')
f.write(f'{[ int(x) for x in [Urf]]} V\n')
f.write(f'{cond_zero_name} : {elem_0} élément(s)\n')
f.write(f'{try_subdir} \n\n')

f.write(f'> X, Y, Z : Demi-axes moyens du nuage Ca+ [µm]\n')
f.write(f'> d_moy : Distance moyenne entre les ions du nuage Ca+ [µm]\n')
f.write(f' Udc  :     X      Y      Z      d_moy\n')
for j in range(elem_2):
    f.write(f'{Udc[j]:.2f}  :  {[ float(float_formatter(mean(dim_nu[:,my_order[j],x])*1000)) for x in [0,1,2]]}  {dist_ions[j]*1e6:.3f}\n')
    
# f.write(f'> Molécule géante - position max\n')
# f.write(f' Udc  :    rx[nm]      ry[nm]     rz[mm]\n')
# for j in range(elem_2):
#     f.write(f'{Udc[j]:.2f}  :  {[ float_formatter(r_x[my_order[j]]), r_y[my_order[j]], r_z[my_order[j]] ]}\n')

f.close()

NameError: name 'N' is not defined

In [125]:
my_q = [0.5,0.5+0.05/3,0.5+2*0.05/3,0.55,0.55+0.05/3,0.55+2*0.05/3,0.6,0.6+0.05/3,0.6+2*0.05/3, 0.65, 0.65+0.05/3, 0.65+0.1/3, 0.7] # [0.6, 0.625, 0.650, 0.675, 0.7]
print(my_q)
coucou = []
figure('pouet')
print('q','   ','Vrf')
for k,l in enumerate(my_q):
    coucou.append(tick_function_q(l))
    print(f'{l:2.5f}',f'{tick_function_q(l):2.2f}')
scatter(my_q,coucou)

[0.5, 0.5166666666666667, 0.5333333333333333, 0.55, 0.5666666666666668, 0.5833333333333334, 0.6, 0.6166666666666667, 0.6333333333333333, 0.65, 0.6666666666666667, 0.6833333333333333, 0.7]
q     Vrf
0.50000 53.84
0.51667 55.64
0.53333 57.43
0.55000 59.23
0.56667 61.02
0.58333 62.82
0.60000 64.61
0.61667 66.41
0.63333 68.20
0.65000 70.00
0.66667 71.79
0.68333 73.59
0.70000 75.38
